
<left>
    <img src="http://epecora.com.br/DataFiles/PPGOLD_logo.png" width="300" alt="PPGOLD Data Science: INTRO"  />
</left>

# *Data Science - PPGOLD*


---


## Case de distribuição de e-commerce



# **Verificar a versão do Python e importa as bibliotecas básicas**

In [ ]:
# check Python version
!python -V

import pandas as pd # download library to read data into dataframe
import numpy as np # import numpy library
import matplotlib.pyplot as plt # plots
import collections
import random # library for random number generation

#Importar Arquivo de dados

Inclua um CSV com os dados a serem analisados
 * Incluindo cabeçalho
 * Separação de campos delimitado por *,*
 * Separação de linha usando *nova linha*

 O arquivo tem mais de 6000 registros, pode levar alguns segundos.

In [ ]:
print("Data read into dataframe!") # takes about 30 seconds

file_path = "Distribution_12k.csv"

starsData = pd.read_csv(file_path)
starsData.head()

# Padronização do DataTable


## Renomear a primeira coluna para "stars"

Para se adequar ao código

In [ ]:
# fix name of the column displaying the Stars
column_names = starsData.columns.values
print(starsData.columns.values)
column_names[0] = "stars"
starsData.columns = column_names
print(column_names)


# Verificando os dados

## Em um contexto real, deveríamos verificar cada coluna minuciosamente procurando por valores fora dos padrões. Mas aqui vamos "verificar" somente a coluna "stars"

### Exemplos mais comuns de valores fora dos padrões são: 

* Valores numéricos fora do intervalo, por exemplo quantidades de estrelas igual a 15, quando devia ser de 1 a 5.
* Texto no local de valores numéricos
* Valores "nan" Not a Number
* Para valores de categorias, no nosso exemplo os produtos são P1, P2, P3; Caso apareca um "Produto 1" precisa ser uniformizado


In [ ]:
starsData["stars"].value_counts() # frequency table

print(starsData["stars"].mean())

c = collections.Counter(starsData["stars"])
c = sorted(c.items())
print(c)

stars_num = [i[0] for i in c]
freq = [i[1] for i in c]

f, ax = plt.subplots()

plt.bar(stars_num, freq)

plt.title("Stars Frequency")
plt.xlabel("Stars")
plt.ylabel("Frequency")

plt.show()

print(stars_num)

# Existem dados fora do intervalo de estrelas [1,5] , alguns valores são 6 outros são NAN

##Limpar dados de estrelas fora do intervalo [1,5], excluindo os registros

In [ ]:
starsData = starsData[starsData["stars"] <= 5]
starsData = starsData[starsData["stars"] >= 1]

starsData = starsData[starsData["FloorCD"] <= 4] # aproveitei e padronizei outros dados numéricos também
starsData = starsData[starsData["FloorCD"] >= 0]

starsData = starsData[starsData["RoutDeliveryOrder"] <= 6]
starsData = starsData[starsData["RoutDeliveryOrder"] >= 1]

starsData["stars"].value_counts() # frequency table

print(starsData["stars"].mean())

c = collections.Counter(starsData["stars"])
c = sorted(c.items())
print(c)

stars_num = [i[0] for i in c]
freq = [i[1] for i in c]

f, ax = plt.subplots()

plt.bar(stars_num, freq)

plt.title("Stars Frequency")
plt.xlabel("Stars")
plt.ylabel("Frequency")

plt.show()

print(stars_num)

starsData = starsData.reset_index(drop=True) # É importante refazer o indice do dataFrame, apeos excluir linhas, pois pode gerar problema no SciKit Learn


#Outras inconsistências nos dados

### Note a coluna Driver_F_CD, ela só tem os valores "D01" ... "D06" mas não sabemos se o motorista D01 pode trabalhar no CD1, CD2 ou CD3 ? Isso não te parece no mínimo esquisito ?

### Fica como Lição de casa você pensar em uma alternativa


In [ ]:
ct = pd.crosstab(index=starsData['CD_name'], columns=starsData['DriverF_CD'])

ct.plot.bar(stacked=True)
plt.legend(title='Driver')

plt.show()

print(ct)


# Começando a análise

## Importar as bibliotecas: 
* SKLEARN
* graphviz
* matplotlib


In [ ]:
# import decision trees scikit-learn libraries
%matplotlib inline
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix # Não usaremos essas métricas neste exemplo

import matplotlib.pyplot as plt

#!conda install python-graphviz --yes
import graphviz

from sklearn.tree import export_graphviz

import itertools

# Separar o dataFrame em variáveis dependentes e independentes

In [ ]:
# split the DataFrame
stars = starsData["stars"] # Só a coluna "stars"
stars_class = stars[:]
variables = starsData.iloc[:,1:] # Todas as outras colunas com excessão da "stars"


stars_class.head()

In [ ]:
variables.head()

# Adequação das variáveis categóricas em colunas binárias - OneHotEncoder




In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse= False)

prod_encoded = pd.DataFrame (enc.fit_transform(variables[['Product']])).astype(int)
prod_encoded.columns = enc.get_feature_names(['Product'])

origin_encoded = pd.DataFrame (enc.fit_transform(variables[['Origin']])).astype(int)
origin_encoded.columns = enc.get_feature_names(['Origin'])

CD_encoded = pd.DataFrame (enc.fit_transform(variables[['CD_name']])).astype(int)
CD_encoded.columns = enc.get_feature_names(['CD_name'])

Destination_encoded = pd.DataFrame (enc.fit_transform(variables[['Dest']])).astype(int)
Destination_encoded.columns = enc.get_feature_names(['Dest'])

TruckType_encoded = pd.DataFrame (enc.fit_transform(variables[['TruckType']])).astype(int)
TruckType_encoded.columns = enc.get_feature_names(['TruckType'])

#FloorCD_encoded = pd.DataFrame (enc.fit_transform(variables[['FloorCD']])).astype(int)
#FloorCD_encoded.columns = enc.get_feature_names(['FloorCD'])

TruckType2_encoded = pd.DataFrame (enc.fit_transform(variables[['TruckType2']])).astype(int)
TruckType2_encoded.columns = enc.get_feature_names(['TruckType2'])

DriverCD_C_encoded = pd.DataFrame (enc.fit_transform(variables[['DriverCD_C']])).astype(int)
DriverCD_C_encoded.columns = enc.get_feature_names(['DriverCD_C'])

var1 = pd.concat([prod_encoded, origin_encoded, CD_encoded, Destination_encoded, TruckType_encoded, TruckType2_encoded, DriverCD_C_encoded, variables["RoutDeliveryOrder"], variables["FloorCD"]], axis=1)

var1.head()




## DecisionTreeClassifier e os seus Parâmetros

O algoritimo do DecisionTreeClassifier tem a tendência natural de fazer um OVERFITING, para ajustar isso modificamos os parâmetros:

* max_depth : Profundidade máxima da árvore
* min_samples_split : quantidade mínima de amostras para se dividir um nó
* min_samples_leaf : Quantidade mínima de amostras em um nó folha (nó da base da árvore)

In [ ]:
delivery_tree = tree.DecisionTreeClassifier(max_depth = 4, min_samples_split = 2, min_samples_leaf = 1)
delivery_tree.fit(var1, stars_class)

print("Decision tree model saved to delivery_tree!")
print(delivery_tree)
print(var1.columns.values)

# Resultados em formato gráfico são sempre melhores ;)

In [ ]:
export_graphviz(delivery_tree,
                feature_names=list(var1.columns.values),
                out_file="delivery_tree.dot",
                class_names=np.unique(stars_class.astype(str)),
                filled=True,
                rounded=True,
                node_ids=True,
                special_characters=True,
                impurity=False,
                label="all",
                leaves_parallel = False)

with open("delivery_tree.dot") as delivery_tree_image:
    delivery_tree_graph = delivery_tree_image.read()
    
graphviz.Source(delivery_tree_graph)

# *Modificar a variável dependente para (bad, average, good) ao invés de [1,5]*

## Atenção: na saída gráfica os valores do vetor Value estarão em ordem alfabeetica, ou seja, (average, bad, good)

In [ ]:
stars_class.loc[stars <= 2] = "bad"
stars_class.loc[stars == 3] = "average"
stars_class.loc[stars >= 4] = "good"

delivery_tree = tree.DecisionTreeClassifier(max_depth = 4, min_samples_split = 2, min_samples_leaf = 1)
delivery_tree.fit(var1, stars_class)

print("Decision tree model saved to delivery_tree!")
print(delivery_tree)
print(var1.columns.values)

export_graphviz(delivery_tree,
                feature_names=list(var1.columns.values),
                out_file="delivery_tree.dot",
                class_names=np.unique(stars_class.astype(str)),
                filled=True,
                rounded=True,
                node_ids=True,
                special_characters=True,
                impurity=False,
                label="all",
                leaves_parallel = False)

with open("delivery_tree.dot") as delivery_tree_image:
    delivery_tree_graph = delivery_tree_image.read()
    
graphviz.Source(delivery_tree_graph)

## Parece que o P7 é chave nesta árvore não?

In [ ]:
stars_class_p7 = stars_class[var1["Product_P7"]==1]
var1_p7 = var1[var1["Product_P7"]==1]


delivery_tree = tree.DecisionTreeClassifier(max_depth = 4, min_samples_split = 2, min_samples_leaf = 1)
delivery_tree.fit(var1_p7, stars_class_p7)

print("Decision tree model saved to delivery_tree!")
print(delivery_tree)
print(var1.columns.values)

export_graphviz(delivery_tree,
                feature_names=list(var1.columns.values),
                out_file="delivery_tree.dot",
                class_names=np.unique(stars_class.astype(str)),
                filled=True,
                rounded=True,
                node_ids=True,
                special_characters=True,
                impurity=False,
                label="all",
                leaves_parallel = False)

with open("delivery_tree.dot") as delivery_tree_image:
    delivery_tree_graph = delivery_tree_image.read()
    
graphviz.Source(delivery_tree_graph)

## Parece que o andar onde alocamos os P07 também tem um papel importante né?

In [ ]:
stars_class_route56 = stars_class_p7[var1_p7["FloorCD"]>=3]
var1_route456 = var1_p7[var1_p7["FloorCD"]>=3]


delivery_tree = tree.DecisionTreeClassifier(max_depth = 4, min_samples_split = 2, min_samples_leaf = 1)
delivery_tree.fit(var1_route456, stars_class_route56)

print("Decision tree model saved to delivery_tree!")
print(delivery_tree)
print(var1.columns.values)

export_graphviz(delivery_tree,
                feature_names=list(var1.columns.values),
                out_file="delivery_tree.dot",
                class_names=np.unique(stars_class.astype(str)),
                filled=True,
                rounded=True,
                node_ids=True,
                special_characters=True,
                impurity=False,
                label="all",
                leaves_parallel = False)

with open("delivery_tree.dot") as delivery_tree_image:
    delivery_tree_graph = delivery_tree_image.read()
    
graphviz.Source(delivery_tree_graph)

## Autor

<a href="https://www.linkedin.com/in/eduardopecora/" target="_blank">Eduardo Pecora</a>

## Log de modificações

| Data | Versão | Modificado por | Descrição |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 04-04-2021       | 1.0     | Eduardo Pecora    | Inicial               |
| 03-11-2021       | 1.1     | Eduardo Pecora    | Estrutura Aula        |

<hr>

